# basic
本小节将会介绍使用pytorch的基本操作和流程

In [2]:
import torch
import numpy

### 1. 一个完整的机器学习流程

1. 导入数据，数据预处理
2. 创建数据迭代器
3. 定义/导入模型
4. 定义损失函数
5. 定义优化算子
6. 训练
7. 保存模型

### 2. tensor基本操作

In [3]:
# 从数据构建tensor
a = torch.Tensor([[3,4,5], [4,5,6]])
a

tensor([[ 3.,  4.,  5.],
        [ 4.,  5.,  6.]])

In [4]:
# 获取tensor的尺寸
a.size()

torch.Size([2, 3])

In [5]:
# 索引
a[0]

tensor([ 3.,  4.,  5.])

In [6]:
# 改变形状
a.view(3, 2)

tensor([[ 3.,  4.],
        [ 5.,  4.],
        [ 5.,  6.]])

In [7]:
# 如果tensor中只含有一个值，获取
a = torch.Tensor([1])
a.item()

1.0

In [8]:
# 改变tensor类型
a.type(torch.LongTensor)

tensor([ 1])

### 3. tensor 与 numpy的转换

In [9]:
# 创建numpy数组
a_np = numpy.array([1,2,3])
a_np

array([1, 2, 3])

In [10]:
# numpy -> tensor
a_tensor = torch.from_numpy(a_np)
a_tensor

tensor([ 1,  2,  3])

In [11]:
# tensor -> numpy
a_np = a_tensor.numpy()
a_np

array([1, 2, 3])

### 4. 数据迭代器

In [12]:
from torch.utils import data

In [13]:
# 创建x
x = torch.linspace(1, 10, 10)
x

tensor([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.])

In [14]:
# 创建y
y = torch.linspace(10, 1, 10)
y

tensor([ 10.,   9.,   8.,   7.,   6.,   5.,   4.,   3.,   2.,   1.])

In [15]:
# 创建迭代器
data_set = data.TensorDataset(x, y)
data_iter = data.DataLoader(
    dataset=data_set,
    batch_size=4,
    drop_last=True,
    shuffle=True
)
for epoch in range(4):
    for i, (x_batch, y_batch) in enumerate(data_iter):
        # ================= #
        #  此处可放入训练代码  #
        # ================= #
        print('epochs:', epoch, '  batches:', i, '  x_batch:', x_batch, '  y_batch:', y_batch)

epochs: 0   batches: 0   x_batch: tensor([ 9.,  6.,  7.,  1.])   y_batch: tensor([  2.,   5.,   4.,  10.])
epochs: 0   batches: 1   x_batch: tensor([ 8.,  3.,  2.,  4.])   y_batch: tensor([ 3.,  8.,  9.,  7.])
epochs: 1   batches: 0   x_batch: tensor([ 3.,  8.,  4.,  5.])   y_batch: tensor([ 8.,  3.,  7.,  6.])
epochs: 1   batches: 1   x_batch: tensor([ 10.,   6.,   1.,   7.])   y_batch: tensor([  1.,   5.,  10.,   4.])
epochs: 2   batches: 0   x_batch: tensor([ 2.,  3.,  1.,  6.])   y_batch: tensor([  9.,   8.,  10.,   5.])
epochs: 2   batches: 1   x_batch: tensor([ 5.,  4.,  9.,  7.])   y_batch: tensor([ 6.,  7.,  2.,  4.])
epochs: 3   batches: 0   x_batch: tensor([  8.,   1.,   6.,  10.])   y_batch: tensor([  3.,  10.,   5.,   1.])
epochs: 3   batches: 1   x_batch: tensor([ 2.,  7.,  9.,  3.])   y_batch: tensor([ 9.,  4.,  2.,  8.])


### 5. 两种定义模型的方式
 - class
 - nn.Sequential
 
以上两种方法等效

In [16]:
from torch import nn

In [17]:
# class方式
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 10)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(10, 20)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.fc2(x)
        return x
net = Net()
net    

Net(
  (fc1): Linear(in_features=5, out_features=10, bias=True)
  (tanh): Tanh()
  (fc2): Linear(in_features=10, out_features=20, bias=True)
)

In [18]:
# nn.Sequential方式
net = nn.Sequential(
    nn.Linear(5, 10),
    nn.Tanh(),
    nn.Linear(10, 20)
)
net

Sequential(
  (0): Linear(in_features=5, out_features=10, bias=True)
  (1): Tanh()
  (2): Linear(in_features=10, out_features=20, bias=True)
)

In [19]:
# 查看模型参数数量
count = 0
for i in net.parameters():
    count += i.nelement()
count

280

### 6. 调用gpu

In [20]:
# 查看gpu是否可用
torch.cuda.is_available()

True

In [21]:
# 上传gpu
data = torch.Tensor([1,2,3])
data = data.cuda()
data

tensor([ 1.,  2.,  3.], device='cuda:0')

In [22]:
# cpu
data = data.cpu()
data

tensor([ 1.,  2.,  3.])

### 7. 定义损失函数

In [23]:
# 交叉熵： softmax + log + '-'
criterion = nn.CrossEntropyLoss()
criterion

CrossEntropyLoss()

### 8. 定义优化算子

In [24]:
# 随机梯度下降
from torch import optim
optimizer = optim.SGD(net.parameters(), lr=0.0001)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0001
    momentum: 0
    nesterov: False
    weight_decay: 0
)

### 9. 训练

In [25]:
# 训练套路
epochs = EPOCH
data_iter = DATA_ITER
for epoch in range(epochs):
    for i, (x_batch, y_batch) in enumerate(data_iter):
        
        # ==================== #
        # 处理 data_batch 数据  #
        # ==================== #
        
        # 训练
        result = net(x_batch)
        loss = criterion(result, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

NameError: name 'EPOCH' is not defined

### 10. 保存与载入

特别注意：有些优化器的参数也需要save和load

In [36]:
# 保存模型
torch.save(net, 'model.pkl')

/home/xy/anaconda3/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [37]:
# 载入模型
net = torch.load('model.pkl')
net

Net(
  (fc1): Linear(in_features=5, out_features=10, bias=True)
  (tanh): Tanh()
  (fc2): Linear(in_features=10, out_features=20, bias=True)
)

In [38]:
# 保存参数
torch.save(net.state_dict(), 'model_dict.pkl')

In [39]:
# 载入参数
net = Net()
net.load_state_dict(torch.load('model_dict.pkl'))
net

Net(
  (fc1): Linear(in_features=5, out_features=10, bias=True)
  (tanh): Tanh()
  (fc2): Linear(in_features=10, out_features=20, bias=True)
)